In [36]:
import numpy as np
from tokenizers import Tokenizer, models
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordLevelTrainer
import pandas as pd
from IPython.display import display, Markdown

text_example = "Alice had no idea what to do"

tokenizer = Tokenizer(models.WordLevel(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = WordLevelTrainer(special_tokens=["[UNK]"])
tokenizer.train_from_iterator([text_example], trainer=trainer)
output = tokenizer.encode(text_example)

vocab = tokenizer.get_vocab()

vocab_df = pd.DataFrame(list(vocab.items()), columns=["Token", "ID"])

sentence_df = pd.DataFrame({ "Token": output.tokens, "ID": output.ids })
display(Markdown("## Sentence with token IDs"))
display(Markdown("Step 1 - Construct a word based vocab from the sentence"))
display(sentence_df)


## Sentence with token IDs

Step 1 - Construct a word based vocab from the sentence

,Token,ID
0,Alice,1
1,had,3
2,no,5
3,idea,4
4,what,7
5,to,6
6,do,2


In [81]:
import pandas as pd
import torch.nn as nn
import torch
embedding_dim = 3

# Generate a simple random 3 dim embedding for the vocab
embedding = nn.Embedding(tokenizer.get_vocab_size(), embedding_dim)

sequence_ids = torch.tensor(output.ids)
sequence_embedding_vectors = embedding(sequence_ids)

# For pandas display purposes
seq_ev_np = sequence_embedding_vectors.detach().numpy()

display(Markdown("## Sentence with random embedding vectors"))
pd.DataFrame(sequence_embedding_vectors.detach().numpy(), index=output.tokens)

## Sentence with random embedding vectors

,0,1,2
Alice,1.614764,0.190990,-0.733677
had,-1.580093,-0.060611,0.430378
no,1.849383,0.404539,-1.158705
idea,0.617203,-0.033330,-0.222188
what,-0.780946,0.171008,0.555985
to,-0.824286,-0.299747,-0.251405
do,1.484740,-0.981164,0.959198


In [84]:
import numpy as np

# Max span length, i.e. 1 and 2 word spans
L = 2

span_embeddings = []
span_word_indices = []
for i in range(len(output.ids)):
    for j in range(i, min(i + span_length, len(output.ids))):
        span_embeddings.append(sequence_embedding_vectors[i:j+1])
        span_word_indices.append([i, j])

padded = torch.zeros(len(span_word_indices), L, embedding_dim)
mask = torch.zeros(len(span_word_indices), L, 1)

for i_seq, seq in enumerate(span_word_indices):
    i, j = seq
    p = j-i + 1
    padded[i_seq, :p] = sequence_embedding_vectors[seq[0]:seq[1]+1]
    mask[i_seq, :p] = 1

sums = (padded * mask).sum(dim=1)
counts = mask.sum(dim=1)
means = sums / counts

for i_seq, seq in enumerate(span_word_indices):
    display(Markdown(f"## Span: {i_seq}"))
    i, j = seq
    p = j-i + 1

    tokens = ['Masked'] * L
    tokens[:p] = output.tokens[i:j+1]
    sequence_vectors = padded[i_seq, :L].detach().numpy()
    display(pd.DataFrame(padded[i_seq, :L].detach().numpy(), index=tokens))
    print(f"Span: '{output.tokens[i:j+1]}' - Mean Embedding: {means[i_seq].detach().numpy()}")

## Span: 0

,0,1,2
Alice,1.614764,0.19099,-0.733677
Masked,0.000000,0.00000,0.000000


Span: '['Alice']' - Mean Embedding: [ 1.6147645   0.19099018 -0.733677  ]


## Span: 1

,0,1,2
Alice,1.614764,0.190990,-0.733677
had,-1.580093,-0.060611,0.430378


...

,0,1,2
to,-0.824286,-0.299747,-0.251405
do,1.484740,-0.981164,0.959198


Span: '['to', 'do']' - Mean Embedding: [ 0.33022678 -0.6404553   0.35389674]


## Span: 12

,0,1,2
do,1.48474,-0.981164,0.959198
Masked,0.00000,0.000000,0.000000


Span: '['do']' - Mean Embedding: [ 1.4847395  -0.981164    0.95919824]
